# Apresentação - Arquitetura de Dados Health Tech


## Objetivo: 
#### Desenvolver toda a infraestrutura necessária para que os dados gerados por um chatbot, chamado Lumia, seja acessado da por analistas de dados, para assim desenvolverem dashboards que consigam responder dúvidas do time de negócios.

### Considerações iniciais

#### Habilidades Lumia:
1. agendamentos
2. consulta de cobertura de convênios médicos
3. informações sobre preparo para procedimentos e exames
4. retorna resultados e laudos prontos
5. avaliação da experiência

#### O que foi possível inferir:
1.o bot já está conectado a um banco de dados, pois algumas dessas capacidades exigem isso.
<br>2.o bot captura informações como os dados pessoais do paciente e exame desejado

#### Conclusões:
1.Não será necessário fazer o ETL entre o bot e o banco de dados, uma vez que essas funcionalidades exigem a conexão com um DB.
<br>2.Logo, o meu trabalho será realizar as transformações de acordo com a demanda dos analistas, ~talvez sendo necessário levar os dados a um data mart.~ mas como não sei qual arquitetura é utilizada pela empresa, vou montar um data lake, um data mart e a pipeline para orquestrar todos os dados.
<br>__3.As informações da Lumia que podem ser úteis aos analistas de dados:__ 
 1. __Agendadamentos      ->__ valor movimentado por área médica, quantidade de consultas marcadas por área e faixa etária de quem utiliza a ferramenta. Quais horários são mais procurados? 
 2. __Avaliação           ->__ Estão dando feedback? Qual a nota que estão dando para a experiência?
 3. __Resultados/Laudos   ->__ Quantos pacientes estão utilizando essa ferramenta? Existe variação do uso da ferramenta por área?
 4. __Geral               ->__ O Lumia está sendo utilizado com qual finalidade? Quando estão utilizando a ferramenta?
 
<br>4.Percebe-se que todas as informações são geradas pelo próprio bot, então vou considerar que tudo que levarei ao data mart está em apenas um banco de dados.
<br>4.1.Pensei no desejo de comparar a utilização do Lumia com os meios tradicionais de comunicação de um hospital/clínica, mas ao fazer uma pesquisa mais aprofundada da empresa em questão, percebi que muito é falado sobre BI, então acredito que essa análise já deve ter sido realizada por outro time.
<br>5.Como não possuo a expertise para desenvolver um bot na Azure, vou simular o retorno que o bot produz atráves de arquivos JSON que um script em Python produzirá.

## Arquitetura construída

<img src='img_projeto_lumia/ArquiteturaLumia.png'>

## Explicação das tecnologias usadas

1. __Azure Cosmos DB (API NOSQL):__ banco de dados para dados semi-estruturados, possui alta disponibilidade para que o time responsável pelo Lumia consiga monitorar os dados produzidos por ele quase em tempo real.
<br><br>
2. __Copy data:__ a camada bronze de um data lake deve armazenar os dados da forma mais crua possível, então utilizei essa ferramenta do Azure Synapse pois não queria alterar os dados de nenhuma forma nessa etapa, apenas replicá-lo. <br><br>
3. __Azure Data Lake Storage Gen2:__ tecologia Azure de baixo custo e com a habilidade de armazenar dados estruturados e não-estrutudados, ideal a camada bronze de um data lake.<br><br>
4. __Data Flow:__ foi utilizado para realizar alguns tratamentos nos dados de acordo com as normas da empresa, como o nome das colunas no formato 'snake case' e datas no formato 'datetime'.<br><br>
5. __Dedicated SQL Pool:__ para a camada silver e data mart, utilizei essa tecnologia da Azure para armazenar dados estruturados para permitir insights de forma rápida e robusta.<br><br>
6. __Stored Procedures:__ para realizar transformações mais complexas e permitir uma manutenção do código caso necessário, utilizei esse tecnologia para a última etapa da pipeline, uma vez que SQL é uma linguagem extretamente comum entre os engenheiros de dados, o que facilitaria a manutenção caso outro time fosse envolvido. <br><br>
7. __Power BI:__ ferramenta acessível e possivelmente a mais utilizada para desenvolvimento de dashboards, levei em conta novamente a questão da manutenção por terceiros.

# Aprensentação visual do projeto

### Script Python

In [2]:
import random as rd
import string

alfabeto = list(string.ascii_lowercase)

listaDesejo = ["Agendamento", "Feedback", "Resultados"]
listaEspecialidade = {"Neurologia":"70", "Cardiologia":"84.50", "Ortopedia":"50.30", "Geriatria":"90.20", "Pediatria":"80"}
listaIdade =  rd.randint(18,70)
listaNome = rd.choice(alfabeto) + rd.choice(alfabeto) + rd.choice(alfabeto) 
listaFeedback = rd.randint(0,1)
listaDias = [x if x>9 else "0"+str(x) for x in range(1,29)]
listaMeses = [x if x>9 else "0"+str(x) for x in range(1,13)]
listaHoras = [x if x>9 else "0"+str(x) for x in range(0,25)]
listaMinutosSegundos = [x if x>9 else "0"+str(x) for x in range(0,60)]

jsonFinal = '['

#  2004-05-23T14:25:10
for x in range(1,1001):

        
    desejo = rd.choice(listaDesejo)
    especialidade = rd.choice(list(listaEspecialidade.keys()))
    valor = listaEspecialidade[especialidade]
    idade = rd.randint(18,70)
    nome = rd.choice(alfabeto) + rd.choice(alfabeto) + rd.choice(alfabeto) 
    feedback = rd.randint(0,1)
    ano = rd.randint(2022,2023)
    mes = rd.choice(listaMeses)
    dia = rd.choice(listaDias)
    hora = rd.choice(listaHoras)
    minuto = rd.choice(listaMinutosSegundos)
    segundo = rd.choice(listaMinutosSegundos)
    dataAtual = f'{ano}-{mes}-{dia}T{hora}:{minuto}:{segundo}'
    #dataAgendada = f'{rd.choice(listaDias)}/{rd.choice(listaMeses)}/{rd.randint(2022,2023)}'
    
    strTemp = f'''
    "id": "{x}",
    "DesejoPaciente": "{desejo}",
    "Especialidade": "{especialidade}",
    "Orcamento": "{valor}",
    "IdadePaciente": "{idade}",
    "NomePaciente": "{nome}",
    "DataAtual": "{dataAtual}",
    "DataAgendada": "{dataAtual}",
    "Feedback": "{feedback}" '''
    
    if feedback == 1:
        strTemp += f''',
    "NotaProfissional": "{rd.randint(1,10)}",
    "NotaGeral": "{rd.randint(1,10)}"'''
        
    jsonFinal += '\n{' + strTemp + '\n},'

jsonFinal = jsonFinal[:-1] + '\n]'

#print(jsonFinal)

#with open("sample.json", "w") as sampleJson:
#    sampleJson.write(jsonFinal)

## JSON gerado

<img src='img_projeto_lumia/sampleJson.png'>

## Dados no CosmosDB

<img src='img_projeto_lumia/dataInsideCosmosDB.png'>

## Pipeline Completa

<img src='img_projeto_lumia/PipelineCompleta.png'>

## Etapas do Data Flow Silver

<img src='img_projeto_lumia/EtapasDataFlowSilver.png'>

## Schema da tabela Silver

<img src='img_projeto_lumia/SchemaSilver.png'>

## Exemplo de procedure utilizada

<img src='img_projeto_lumia/ExemploDeProcedureUtilizada.png'>


## Dashboard no Power Bi ( apenas para mostrar que tudo funcionou corretamente )

<img src='img_projeto_lumia/TelaSimplesPowerBI.png'>